<a href="https://colab.research.google.com/github/google/jax-md/blob/master/notebooks/talk_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Import & Util

!pip install -q git+https://www.github.com/google/jax
!pip install -q git+https://www.github.com/google/jax-md
!pip install dm-haiku

import jax.numpy as np
from jax import device_put
from jax.config import config
# TODO: Uncomment this and enable warnings when XLA bug is fixed.
import warnings; warnings.simplefilter('ignore')
# config.update('jax_enable_x64', True)
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

import warnings
warnings.simplefilter("ignore")

sns.set_style(style='white')
background_color = [56 / 256] * 3
def plot(x, y, *args):
  plt.plot(x, y, *args, linewidth=3)
  plt.gca().set_facecolor([1, 1, 1])
def draw(R, **kwargs):
  if 'c' not in kwargs:
    kwargs['color'] = [1, 1, 0.9]
  ax = plt.axes(xlim=(0, float(np.max(R[:, 0]))), 
                ylim=(0, float(np.max(R[:, 1]))))
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.set_facecolor(background_color)
  plt.scatter(R[:, 0], R[:, 1],  marker='o', s=1024, **kwargs)
  plt.gcf().patch.set_facecolor(background_color)
  plt.gcf().set_size_inches(6, 6)
  plt.tight_layout()
def draw_big(R, **kwargs):
  if 'c' not in kwargs:
    kwargs['color'] = [1, 1, 0.9]
  fig = plt.figure(dpi=128)
  ax = plt.axes(xlim=(0, float(np.max(R[:, 0]))),
                ylim=(0, float(np.max(R[:, 1]))))
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.set_facecolor(background_color)
  s = plt.scatter(R[:, 0], R[:, 1], marker='o', s=0.5, **kwargs)
  s.set_rasterized(True)
  plt.gcf().patch.set_facecolor(background_color)
  plt.gcf().set_size_inches(10, 10)
  plt.tight_layout()
def draw_displacement(R, dR):
  plt.quiver(R[:, 0], R[:, 1], dR[:, 0], dR[:, 1], color=[1, 0.5, 0.5])

# Progress Bars

from IPython.display import HTML, display
import time

def ProgressIter(iter_fun, iter_len=0):
  if not iter_len:
    iter_len = len(iter_fun)
  out = display(progress(0, iter_len), display_id=True)
  for i, it in enumerate(iter_fun):
    yield it
    out.update(progress(i + 1, iter_len))

def progress(value, max):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 45%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

# Data Loading

!wget -O silica_train.npz https://www.dropbox.com/s/3dojk4u4di774ve/silica_train.npz?dl=0
!wget https://github.com/google/jax-md/blob/master/examples/models/si_gnn.pickle?raw=true

import numpy as onp

with open('silica_train.npz', 'rb') as f:
  files = onp.load(f)
  Rs, Es, Fs = [device_put(x) for x in (files['arr_0'], files['arr_1'], files['arr_2'])]
  Rs = Rs[:10]
  Es = Es[:10]
  Fs = Fs[:10]
  test_Rs, test_Es, test_Fs = [device_put(x) for x in (files['arr_3'], files['arr_4'], files['arr_5'])]
  test_Rs = test_Rs[:200]
  test_Es = test_Es[:200]
  test_Fs = test_Fs[:200]

## Demo

www.github.com/google/jax-md -> notebooks -> talk_demo.ipynb

### Energy and Automatic Differentiation

$u(r) = \begin{cases}\frac13(1 - r)^3 & \text{if $r < 1$} \\ 0 & \text{otherwise} \end{cases}$

In [ ]:
import jax.numpy as np

def soft_sphere(r):
  return np.where(r < 1, 
                  1/3 * (1 - r) ** 3,
                  0.)

print(soft_sphere(0.5))

In [ ]:
r = np.linspace(0, 2., 200)
plot(r, soft_sphere(r))

We can compute its derivative automatically

In [ ]:
from jax.api import grad

du_dr = grad(soft_sphere)

print(du_dr(0.5))

We can vectorize the derivative computation over many radii

In [ ]:
from jax.api import vmap

du_dr_v = vmap(du_dr)

plot(r, soft_sphere(r))
plot(r, -du_dr_v(r))

### Randomly Initialize a System

In [ ]:
from jax import random

key = random.PRNGKey(0)

particle_count = 128
dim = 2

In [ ]:
from jax_md.quantity import box_size_at_number_density

# number_density = N / V
side_length = box_size_at_number_density(particle_count = particle_count, 
                                         number_density = 1.0, 
                                         spatial_dimension = dim)

R = random.uniform(key, (particle_count, dim), maxval=side_length)
draw(R)

### Displacements and Distances


In [ ]:
from jax_md import space

displacement, shift = space.periodic(side_length)

print(displacement(R[0], R[1]))

In [ ]:
metric = space.metric(displacement)

print(metric(R[0], R[1]))

Compute distances between pairs of points

In [ ]:
displacement = space.map_product(displacement)
metric = space.map_product(metric)

print(metric(R[:3], R[:3]))

### Total energy of a system

In [ ]:
def energy(R):
  dr = metric(R, R)
  return 0.5 * np.sum(soft_sphere(dr))

In [ ]:
print(energy(R))

In [ ]:
print(grad(energy)(R).shape)

### Minimization

In [ ]:
from jax_md.minimize import fire_descent

init_fn, apply_fn = fire_descent(energy, shift)

state = init_fn(R)

while np.max(np.abs(state.force)) > 1e-3:
  state = apply_fn(state)

draw(state.position)

In [ ]:
cond_fn = lambda state: np.max(np.abs(state.force)) > 1e-3

### Making it Fast

In [ ]:
def minimize(R):
  init, apply = fire_descent(energy, shift)

  state = init(R)

  for _ in range(20):
    state = apply(state)

  return energy(state.position)

In [ ]:
%%timeit
minimize(R).block_until_ready()

In [ ]:
from jax.api import jit

# Just-In-Time compile to GPU
minimize = jit(minimize)

In [ ]:
# The first call incurs a compilation cost
minimize(R)

In [ ]:
%%timeit
minimize(R).block_until_ready()

In [ ]:
from jax.lax import while_loop

def minimize(R):
  init_fn, apply_fn = fire_descent(energy, shift)

  state = init_fn(R)
  # Using a JAX loop reduces compilation cost
  state = while_loop(cond_fun=cond_fn,
                     body_fun=apply_fn,
                     init_val=state)

  return state.position

In [ ]:
from jax.api import jit

minimize = jit(minimize)

In [ ]:
R_is = minimize(R)

In [ ]:
%%timeit
minimize(R).block_until_ready()

### Elastic Moduli

In [ ]:
def strain_energy(strain, R):
  dR = np.dot(displacement(R, R), strain)
  dr = space.distance(dR)
  return 0.5 * np.sum(soft_sphere(dr))

In [ ]:
strain_energy(np.eye(2),  R_is)

In [ ]:
from jax.api import hessian

K = hessian(strain_energy)(np.eye(2),  R_is)
print(K.shape)

In [ ]:
print(K)

In [ ]:
from jax_md.quantity import bulk_modulus

B = bulk_modulus(K)
print(B)

In [ ]:
G = K[0, 1, 0, 1]
print(G)

In [ ]:
from functools import partial

@jit
def elastic_moduli(number_density, key):
  # Randomly initialize particles.
  side_length = box_size_at_number_density(particle_count    = particle_count, 
                                           number_density    = number_density, 
                                           spatial_dimension = dim)
  R = random.uniform(key, (particle_count, dim))

  displacement, shift = space.periodic(side_length)
  displacement = space.map_product(displacement)

  # Define an energy function at a specific strain.
  def energy(strain, R):
    dR = displacement(R, R) @ strain
    dr = space.distance(dR)
    return 0.5 * np.sum(soft_sphere(dr))

  # Minimize at no strain.
  init_fn, apply_fn = fire_descent(partial(energy, np.eye(2)), shift)

  state = init_fn(R)
  state = while_loop(cond_fn, apply_fn, state)

  # Compute the elastic constants.
  K = hessian(energy)(np.eye(2), state.position)
  return bulk_modulus(K), K[0, 1, 0, 1]

In [ ]:
number_densities = np.linspace(1.1, 1.6, 40)

elastic_moduli = vmap(elastic_moduli, in_axes=(0, None))
B, G = elastic_moduli(number_densities, key)

plot(number_densities, B)
plot(number_densities, G)

In [ ]:
plot(number_densities, G / B)
plt.ylim([0, 1])

In [ ]:
keys = random.split(key, 5)

elastic_moduli = vmap(elastic_moduli, in_axes=(None, 0))
B, G = elastic_moduli(number_densities, keys)

for b, g in zip(B, G):
  plt.plot(number_densities, g / b)

plot(number_densities, np.mean(G / B, axis=0), 'k')
plt.ylim([0, 1])

### Going Big

In [ ]:
key = random.PRNGKey(0)

particle_count = 128000
side_length = box_size_at_number_density(particle_count    = particle_count, 
                                         number_density    = 1.0, 
                                         spatial_dimension = dim)


R = random.uniform(key, (particle_count, dim)) * side_length

displacement, shift = space.periodic(side_length)

draw_big(R)

In [ ]:
from jax_md.energy import soft_sphere_neighbor_list

neighbor_fn, energy_fn = soft_sphere_neighbor_list(displacement, side_length)

init_fn, apply_fn = fire_descent(energy_fn, shift)

In [ ]:
nbrs = neighbor_fn(R)
print(nbrs.idx.shape)

In [ ]:
from jax.lax import fori_loop

state = init_fn(R, neighbor=nbrs)

@jit
def take_steps(state_and_nbrs):
  def step(i, state_and_nbrs):
    state, nbrs = state_and_nbrs
    nbrs = neighbor_fn(state.position, nbrs)
    state = apply_fn(state, neighbor=nbrs)
    return state, nbrs
  return fori_loop(0, 10, step, state_and_nbrs)

for i in ProgressIter(range(80)):
  new_state, nbrs = take_steps((state, nbrs))

  if nbrs.did_buffer_overflow:
    nbrs = neighbor_fn(state.position)
  else:
    state = new_state

draw_big(state.position)

## Neural Network Potentials

Here is some data we loaded of a 64-atom Silicon system computed using DFT.

In [ ]:
print(Rs.shape)  # Positions
print(Es.shape)  # Energies
print(Fs.shape)  # Forces

In [ ]:
E_mean = np.mean(Es)
E_std = np.std(Es)

print(f'E_mean = {E_mean}, E_std = {E_std}')

In [ ]:
plt.hist(Es)

Setup the system and a Graph Neural Network energy function

In [ ]:
box_size = 10.862
displacement, shift = space.periodic(box_size)

In [ ]:
from jax_md.energy import graph_network

init_fn, energy_fn = graph_network(displacement, r_cutoff=3.0)

In [ ]:
params = init_fn(key, test_Rs[0])
energy_fn(params, test_Rs[0])

In [ ]:
vectorized_energy_fn = vmap(energy_fn, (None, 0))
predicted_Es = vectorized_energy_fn(params, test_Rs)
plt.plot(test_Es, predicted_Es, 'o')

Define a loss function.

In [ ]:
def energy_loss_fn(params):
  return np.mean((vectorized_energy_fn(params, Rs) - Es) ** 2)

def force_loss_fn(params):
  # We want the gradient with respect to the position, not the parameters.
  grad_fn = vmap(grad(energy_fn, argnums=1), (None, 0))
  return np.mean((grad_fn(params, Rs) + Fs) ** 2)

@jit
def loss_fn(params):
  return energy_loss_fn(params) + force_loss_fn(params)

Take a few steps of gradient descent.

In [ ]:
from jax.experimental import optix

opt = optix.chain(optix.clip_by_global_norm(0.01),
                  optix.adam(1e-4))

opt_state = opt.init(params)

@jit
def update(params, opt_state):
  updates, opt_state = opt.update(grad(loss_fn)(params), opt_state)
  return optix.apply_updates(params, updates), opt_state

for i in ProgressIter(range(100)):
  params, opt_state = update(params, opt_state)
  if i % 10 == 0:
    print(f'Loss at step {i} is {loss_fn(params)}')

Now load a pretrained model.

In [ ]:
with open('si_gnn.pickle?raw=true', 'rb') as f:
  params = pickle.load(f)

In [ ]:
from functools import partial
energy_fn = partial(energy_fn, params)

In [ ]:
predicted_Es = vmap(energy_fn)(test_Rs)
plt.plot(test_Es, predicted_Es, 'o')

In [ ]:
from jax_md.quantity import force

force_fn = force(energy_fn)
predicted_Fs = force_fn(test_Rs[1])

plt.plot(test_Fs[1].reshape((-1,)), predicted_Fs.reshape((-1,)), 'o')

This energy can be used in a simulation

In [ ]:
from jax_md.simulate import nvt_nose_hoover
from jax_md.quantity import temperature

K_B = 8.617e-5
dt = 1e-3
kT = K_B * 300 
Si_mass = 2.91086E-3

init_fn, apply_fn = nvt_nose_hoover(energy_fn, shift, dt, kT, tau=3.0)

apply_fn = jit(apply_fn)

In [ ]:
state = init_fn(key, Rs[0], Si_mass, T_initial=300 * K_B)

@jit
def take_steps(state):
  return fori_loop(0, 1000, lambda i, state: apply_fn(state), state)

print('Energy (eV)\tTemperature (K)')
for i in range(10):
  state = take_steps(state)

  print('{:.02f}\t\t\t{:.02f}'.format(
      energy_fn(state.position),
      temperature(state.velocity, Si_mass) / K_B))